A drill to get you into the FDOC data
-------------------------------------

Last time we started looking at Florida's Offender Based Information System. It is essentially a database dump of what the department of corrections considers to be non-sensitive information about every inmate passing through the system since 1997. 

NOTE: These data were pulled last year so were current as of September 2016! This will be important later.

First, I have made the data files available individually as well as bundled for easier downloading. Here is [the web site with the separate files](http://compute-cuj.org/FDOC/) and here is a bundled file you can download to your computer to use [FDOC.tar.gz](http://compute-cuj.org/FDOC/FDOC.tar.gz) -- it is a tar'd (yes, a tape archive) and compressed folder. If you choose to download the data, you change all the read.csv() calls not to URL given, but just to the file specified.

To have a look at these files, we can read in just a few observations and avoid long load times. While there are a number of tables, I'll start with the active inmates and work out from there. 

> - **Inmate_active_root**:  Basic inmate information on active inmates
- **Inmate_active_aliases**: Aliases for these active inmates
- **Inmate_active_offenses_CPS**: Current prison offenses for these active inmates
- **Inmate_active_offenses_prpr**: Prior prison offenses for these active inmates 
- **Inmate_active_detainers**: Detainers for these active inmates
- **Inmate_active_incarhist**: Incarceration history for these active inmates
- **Inmate_active_scarsmarks**: Tattoos for these active inmates

First, we'll work with the "root" file that describes basic statistics about the current inmate population. Here we load in the data and look at the first and the last 5 rows. As we explained in class, if any of the variables look odd or seems coded strangely, you can use the FDOC's  [interactive web applications](http://www.dc.state.fl.us/InmateInfo/InmateInfoMenu.asp) to verify the data.

In [ ]:
root = read.csv("http://compute-cuj.org/FDOC/INMATE_ACTIVE_ROOT.csv",as.is=TRUE)
head(root,5)

In [ ]:
tail(root,5)

With any new table, it's good to look at the top -- the head() -- and the bottom -- the tail() -- and even a few random rows drawn from the table. Remember we can do the latter with sample_n() in dplyr. Execute the code below a few times and get a feel for what the data are showing -- this is a crude "feel" at best, of course.

In [ ]:
library(dplyr)
sample_n(root,5)

**Data frames**

The object "root" is a data frame, R's answer to a table. It has rows (inmates) and columns (aspects of their incarceration). You can get the count of each with dim().

In [ ]:
dim(root)

The inference here is that there are 100,196 inmates currently serving time in the Florida prison system. This data frame has 22 different pieces of data about each inmate. From their BirthDate to their ReceiptDate, the day they were incarcerated. 

The first column is the inmate's DCNumber (also known as their department number). It is a unique ID for each inmate. Despite its appearance in our head() display above, it is not a number, but a character string. The tail() rows above have DCNumbers that start with the letter "Y" (and have ReceiptDates in 2016), while the random sampling turns up a range of letters. 

**1. Do a little research and tell me about the DCNumber. How is it assigned and what information, if any, can you tell about the inmate from the number itself?**

Write answer here...



Now, you can use summary() on the whole data frame to see the kind of data in each column. This is like glimpse() we saw before but with more data for quantitative variables.

In [ ]:
summary(root)

Only "Height" and "Weight" were recognized as numbers (and we have talked about the peculiar way "Height" is stored) and are summarized with some basic statistics like mean and median and the number of missing values. Everything else looks like a character string. When we read in the data into our table "root", we passed the argument "as.is=TRUE". This kept R from working very hard on the character data. In the future, we'll let the program try to guess what kind of data the character strings hold -- dates? general categorical data?

**2. Write down in the cell below, what each column contains and and something about the format it is in. For example, there is a column "Sex" -- how is that coded? What about BirthDate?**

Write your Markdown here




**Joining data frames**

Let's look at another data frame (table) -- the crimes each inmate is convicted of. Read it in, as usual, and call it "offense".

In [ ]:
offense = read.csv("http://compute-cuj.org/FDOC/INMATE_ACTIVE_OFFENSES_CPS.csv",as.is=T)
head(offense,5)

In [ ]:
tail(offense,5)

In [ ]:
dim(offense)

This means there are nearly 400,000 offences in the file for 100,000 inmates, suggesting multiple offenses per inmate. Let's investigate that. This file again has DCNumber as a unique identiier. We can use this piece of information, common to both tables "root" and "offense" to join the two data sets. 

But first, we should boil down the offense data so that each row refers to a separate inmate. We can do this using the group_by() and summarise() commands we saw last time. Oh and the n() function simply counts the number of elements in a group. Last time we saw you could use mean() or median() or sd() to summarize a particular column, but n() just counts how many offenses are assigned to each inmate. 

In [ ]:
numoff = summarise(group_by(offense, DCNumber),count=n())
head(numoff,5)

In [ ]:
tail(numoff,5)

In [ ]:
sample_n(numoff,5)

Look at a few random samples by executing the code above. Do you get a feel for how many offenses inmates are charged with? We will come back to this in the next section. For now, we want to take the counts of offenses and add them to the "root" data set.

Now, if our two tables "root" and "offense" are in sync, then each inmate should have entries in both tables. So, if the "root" file has 100,196 entries, we might expect "numoff"  to have the same number of entries -- one row per inmate, together with the total charges against them. 

Let's check.

In [ ]:
dim(numoff)

We are missing 104 inmates (100,196-100,092 = 104). 

Let's make sure to check which inmates are missing offenses after we join the tables. Now, to combine "root" and "numoff" we have to make some choices. There are various ways to do it. What we want to do is take all the entries in "root" and add data from "numoff" when it exists using "DCNumber" as the key between the two. 

That is, take an inmate from "root", get their "DCNumber", look it up in "numoff" and record the number of offenses they were charged with. When an inmate doesn't have sentencing data in "numoff" (and 104 will not), we'll replace the offense count with an NA or the missing value in "root". OK?

To get a little general for a second, **we want to join data sets "x" and "y" ("x" is said to be on the left, "y" on the right) "by" one or more columns shared by the two data sets.**

In our case, "x" is "root", "y" is "numoff" and "by" is the column "DCNumber" common to both data frames. Our operation of adding data from "numoff" to "root" (from "y" to "x") with NA's if data are missing in "numoff" is called a **left join**. By contrast, we might have only wanted to only return rows in "root" where there are corresponding data in "numoff", dropping entries in "root" rather than introducing NAs -- this is called an **inner join**. 

Here are the other options. Just skim them for now. Each is a function in dplyr.

> - **left_join(x,y,by,...)**: return all rows from x, and all columns from x and y. Rows in x with no match in y will have NA values in the new columns. If there are multiple matches between x and y, all combinations of the matches are returned.
- **inner_join(x,y,by,...)**: return all rows from x where there are matching values in y, and all columns from x and y. If there are multiple matches between x and y, all combination of the matches are returned.
- **right_join(x,y,by,...)**: return all rows from y, and all columns from x and y. Rows in y with no match in x will have NA values in the new columns. If there are multiple matches between x and y, all combinations of the matches are returned.
- **semi_join(x,y,by,...)**: return all rows from x where there are matching values in y, keeping just columns from x. A semi join differs from an inner join because an inner join will return one row of x for each matching row of y, where a semi join will never duplicate rows of x.
- **anti_join(x,y,by,...)**: return all rows from x where there are not matching values in y, keeping just columns from x.
- **full_join(x,y,by,...)**: return all rows and all columns from both x and y. Where there are not matching values, returns NA for the one missing.

Below, we create a new version of "root", having added the data on sentencing from "numoff" in a left_join().

In [ ]:
root = left_join(root,numoff,by="DCNumber")
dim(root)

In [ ]:
head(root,5)

OK so who doesn't have a proper offence count?

Use the dplyr command filter() to pull out those inmates, rows of the new "root," with missing offense count entries. We will use the function is.na() that we have seen before to create TRUE/FALSE data, TRUE if the count of offenses is missing and FALSE otherwise. What do the inmates with missing counts have in common? 

(Oh the options() command at the top tells the notebook to show 100 columns maximum and 300 rows maximum -- you had asked about getting rid of the ellipses last time.)

In [ ]:
options(repr.matrix.max.cols=100,repr.matrix.max.rows=300)

filter(root,is.na(count))

**3. Write your explanation below as to why some inmates are missing an offense count. You might have to research a little to figure out the facts of their incarceration. Where? When? Also, remember that these data are from September 2016 and were current then. So dates like 9/29/16 mean "now" in this data set.**

Write here



Now, your turn. Read in the aliases file, recording all the aliases used by inmates in the current prison population and have a look. We again use read.csv() and try our head() and tail() and dim().

In [ ]:
alias = read.csv("http://compute-cuj.org/FDOC/INMATE_ACTIVE_ALIASES.csv",as.is=TRUE)
head(alias,5)

In [ ]:
tail(alias,5)

In [ ]:
dim(alias)

**4. Create a left join of "root" and "alias" adding the number of aliases associated with each inmate. Are all the inmates accounted for? **

In [ ]:
# put your code here



Write any commentary here



We finish our discussion of data frames with the "numoff" table again. Using dplyr, we can take "numoff" and group_by() "count". This will form groups of inmates having the same number of offenses. We can then count how many inmates are in each group using n(). 

Here we go.

In [ ]:
offcount = summarise(group_by(numoff,count),inmate_total=n())
offcount

This says there are 28,000 or so inmates with just one offense and 6 with 100. Let's look at those in our original "numoff" table.

In [ ]:
filter(numoff,count==100)

... and then go back one more table and find them in "root" using their DCNumber.

In [ ]:
filter(root,DCNumber==361490)

Or in "offense" again using their DCNumber.

In [ ]:
filter(offense,DCNumber==361490)

**5. Pick another total from "offcount" and tell me about the person's offenses.

In [ ]:
# put your code here


Write your comments here



**Vectors**

We can do quite a lot with tables, and the functions in dplyr are well-suited for these operations.  Tables, or data frames, are made up of individual columns, each column representing a different aspect of the place or person or event or object that we've assembled in the table. Each column can be thought of as a **vector**. A vector is just an ordered sequence of values. That means there's a notion of first, second, third and even last. 

We can make vectors using the c() concatenate notation. Here we create three vectors, each with length 5. We can subset them using the square brackets \[  \] but we don't need the commas like we did with data frame. Data frames had rows and columns so we could subset either or both and the comma let us be clear. With a vector we just have one dimension to subset -- which number entry do you want?

In [ ]:
n = c(1,3,4,2,3)
h = c(500,600,510,511,601)
e = c("BRO","GRN","BRO","BLU","BRO")

In [ ]:
length(e)

In [ ]:
length(n)

In [ ]:
h[4]

In [ ]:
e[2]

These columns can be assembled into a data frame, giving each column a name.

In [ ]:
tmp = data.frame(count=n,height=h,eyecolor=e)
tmp

When we use the "$" sign extractor to pull a column from a data table, we invert this process and we get back a vector. Sometimes we want to experiment with transformations to a single vector of data and not carry the full table around with us. 

Here we focus on just the "ReceiptDate" column. The command head() works on data frames and vectors, in both cases just returning the first few items -- rows in a table or elements in a vector.

In [ ]:
arrive = root$ReceiptDate
length(arrive)

In [ ]:
head(arrive,5)

So, "arrive" is a **vector**. It's a sequence of character strings, each representing the date an inmate arrived at an FDOC facility. We see it has 100,196 entries as we would expect, since "root" has 100,196 inmates (rows). You can index various entries with square brackets.

In [ ]:
arrive[100]
arrive[1000]
arrive[100000]

These give us "ReceiptDate" string for the 100th, the 1,000th and the 100,000th inmate. 

**Dates as strings**

Much of what we wanted to do was deal with dates. Right now our dates are encoded as strings. That means they are seen by R as **a sequence of characters**. The three dates above have (count them) 17 charaters each. The look pretty regular. Perhaps we can use that regularity.

Let's make sure they really are regular. First, let's count all the entries and make sure all 100K have 17 characters. For this we use the function nchar().

In [ ]:
nchar('08/21/15 00:00:00')

In [ ]:
nchar(arrive[1])

In [ ]:
nchar(arrive[15000])

Like most functions in R, nchar() can take not just single strings, but a vector of strings, and operate on the whole vector. Here we apply nchar() to the 100K values of ReceiptDates, but show just the first 100 using head().

In [ ]:
head(nchar(arrive),100)

Looks like they're all 17. We can test whether all the values are 17 using the command all(). It takes a vector of TRUE and FALSE values and returns TRUE if they are all TRUE.

In [ ]:
all(nchar(arrive) == 17)

We can do finer tests, for example, making sure the 3rd and 6th characters are slashes. We can pick off pieces of a string with substr(). It takes 3 arguments -- a string (or vector of strings) and then the start of the substring you want and the end. So, the day in '08/21/15 00:00:00' starts at character number 4 and ends at character number 5. Or the first slash starts and ends with character three. Here's how we carve these out.

In [ ]:
substr('08/21/15 00:00:00',4,5)

In [ ]:
substr('08/21/15 00:00:00',3,3)

So, applying the last expression to all of arrive, let's test if the third character is always a slash.

In [ ]:
slash = substr(arrive,3,3)
all(slash=="/")

**6. Now, try this with the second slash. Find what location it should be in and test.**

In [ ]:
# Put code here


To work with dates as strings, we could carve out months and days and years this way. Here's months and days.

In [ ]:
amonths = substr(arrive,1,2)
adays = substr(arrive,4,5)
ayears = substr(arrive,7,8)

head(amonths)

So substr() takes a string and returns a string. Here the months are the strings "10" and "12" rather than the numbers 10 and 12. We can turn them into numbers using the command as.numeric(). Notice that the quotation marks are gone.

In [ ]:
amonths = as.numeric(substr(arrive,1,2))
head(amonths)

Let's do the same for years!

In [ ]:
adays = as.numeric(substr(arrive,4,5))
ayears = as.numeric(substr(arrive,7,8))

head(ayears)

We can use table() to generate a tabulation of the ReceiptDates in "arrive" for each current inmate by year. How many people were admitted in 1980? How many in 2000? Here's table and an associated **barplot**.

In [ ]:
table(ayears)
barplot(table(ayears))

What do you notice? Does this make sense?

It seems that the two digit year has hit a Y2K problem. That is, years with values less than 99 but larger than 46 refer to jail sentences that started in the 1900's. Those from 0 up to 16 refer to sentences that began in the 2000's. To fix that, we want to add 2000 to any year that is 16 or smaller, and 1900 to the rest. 

The command ifelse() does that. It takes three arguments. First is a logical test "is the year smaller or equal to 16?". Then, if it's true it takes one action and if it's false it takes a second. In the code below, if the value of "years" is less than or equal to 16, we return 2000 plus the year. If it's larger than 16, we add 1900 to the ReceiptDate year instead.

We use this command a lot if we just have two choices to toggle between. It's much cleaner than making an "if" statement, for example (which we'll get to shortly).

In [ ]:
ayears = as.numeric(substr(arrive,7,8))
ayears = ifelse(ayears <= 16, 2000+ayears, 1900+ayears)
barplot(table(ayears))

That looks better. But to really understand what's happening, we go back to vectors. Let's make a numeric vector and do some arithmetic.

In [ ]:
x = c(100,5,20,50)
x+1

In [ ]:
x*10

When you add a number (or subtract or divide or multiply) by a numeric vector, R will "vectorize" the operation and apply the sum to the whole thing. So "ayears+1900" is making a new vector of length 100,196, but it is made by adding 1900 to each entry in "ayears". Same goes for "2000+ayears". Then ifelse() just chooses which vector to take an entry from depending on whether "ayears" is smaller or equal to 16. Easy! 

Here we have the code in one place. Note that we could use mutate() and some of the dplyr functions for this, but I find it easier to work with vectors first and understand what the code is doing before adding things back to the data frame.

In [ ]:
# Extract the 2-digit years and correct for 1900 v. 2000
ayears = as.numeric(substr(root$ReceiptDate,7,8))
ayears = ifelse(ayears <= 16, 2000+ayears, 1900+ayears)

# Then paste back the corrected year
root$ReceiptDateFixed = paste(substr(root$ReceiptDate,1,5),ayears,sep="/")

# Have a look
head(root$ReceiptDateFixed)

**7. Create a new "years" vector that corresponds to the BirthDate variable in root. As with ReceiptDate, the small values will refer to the 2000s, while the larger numbers will refer to the 1900s. Where is the line drawn? How can you make sure that people in the small category aren't really really old (as in 1900+something) versus really young (as in 2000+something)? *Hint: Check out the correctional facilities for this group.*<br><br> I'll start you off with a tabulation of the BirthDate years.**

In [ ]:
byears = as.numeric(substr(root$BirthDate,7,8))
table(byears)

In [ ]:
# Put your code here



**Dates objects**

Strings are hard to work with. We can't, for example, tell how old people were when they were admitted to prison. Well, we can say that someone born in 1944 and admitted in 1971 was about 27 years old, but it would depend on the month. Dates are fussy things because months don't all have the same number of days and there are leap years... It's a mess. But we are not the first to encounter dates and others have solved this problem.

The solution in many languages is an "object" of type date. Just like we have numeric data and character data (maybe quantitative and qualitative, say), we can have specialty data types that are much more expressive in the measurements they represent. 

The library "lubridate" is designed for working with dates (making dates easy). There are simple functions like mdy() and ymd() that specify what kind of date string you have. Ours is mdy(), month-day-year. These functions take in strings and return smart date objects. Here we store the dates back in our data frame root.

In [ ]:
library(lubridate)
root$ReceiptDateObj = mdy(root$ReceiptDateFixed)

In [ ]:
head(root$ReceiptDateObj)

Ah, but now the smarts. Take the difference between "ReceiptDateObj"'s for the first two inmates and see what you get!

In [ ]:
root$ReceiptDateObj[1]-root$ReceiptDateObj[2]

Smart! Now, let's make a BirthDateObj, building on what you did above.

In [ ]:
# Extract the 2-digit years and correct for 1900 v. 2000
byears = as.numeric(substr(root$BirthDate,7,8))
byears = ifelse(byears <= 1, 2000+byears, 1900+byears)

# Then paste back the corrected year
root$BirthDateFixed = paste(substr(root$BirthDate,1,5),byears,sep="/")

# Have a look
root$BirthDateObj = mdy(root$BirthDateFixed)
head(root$BirthDateObj)

There are plenty of ways to do arithmetic with dates. The easiest or most expressive just literally divides an interval of time by the duration of a particular unit. Below, we create time intervals that start at birth and end with the "ReceiptDate". We see that these print out as ranges.

In [ ]:
span = interval(root$BirthDateObj,root$ReceiptDateObj)
head(span)

We can finally take those ranges and express them in years or weeks or days (months would be harder because they have a variable number of days). So here we look at the difference in time in units of days, weeks, 21 days and years. The functions ddays(), dweeks() and dyears() produce a duration object that refers to the time you specify. Here it's 1 day, 1 week, 21 days and 1 year. The time intervals are then divided by these durations.

In [ ]:
head(span/ddays(1), 5)
head(span/dweeks(1), 5)
head(span/ddays(21), 5)
head(span/dyears(1), 5)

And so we store the age of the inmate when they were incarcerated, calling it "age". We can have a look with a barplot() table() 1-2 punch :)

In [ ]:
root$age = trunc(span/dyears(1))
barplot(table(root$age))

**Tattoos**

I started working with other data. Many of you mentioned recidivism. That would involve creating a new table from the incarceration history file. You would count the number of times inmates were in prison, the total time spent, or maybe the largest gap betwen prison stays. All of this would use the time functions we learned. 

Some of you also expresed interest in the tattoo data. Below is stub of code that creates a new data frame that can be merged with "root" recording how many tattoos, how many different body parts  and TRUE/FALSE if any were on the face.

In [ ]:
marks = read.csv("http://compute-cuj.org/FDOC/INMATE_ACTIVE_SCARSMARKS.csv",as.is=TRUE)

In [ ]:
sample_n(marks,5)

In [ ]:
table(marks$Location)

In [ ]:
nummark = summarise(group_by(marks, DCNumber),count=n(),type=n_distinct(Location),face="FACE"%in%Location)
head(nummark,5)

We can now compare this to other data sets as we did before, perhaps adding it to "root".